In [ ]:
#@title Datasets: synthetic + MNIST/fMNIST (çoklu ikili) + CIFAR10 + SVHN + **Paper Synthetic A/B/C**
from pathlib import Path
import numpy as np
from sklearn.datasets import make_moons, make_circles, make_blobs
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10

# SVHN (tfds) kurulum/ithalat
try:
    import tensorflow_datasets as tfds
except Exception:
    !pip -q install tensorflow-datasets
    import tensorflow_datasets as tfds

RNG = np.random.default_rng(42)
OUT = Path('data'); OUT.mkdir(exist_ok=True)

def save_npz(X_train, y_train, X_test, y_test, name):
    p = OUT / f"{name}.npz"
    np.savez_compressed(p, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
    print("saved", p, X_train.shape, X_test.shape)

# =========================
# ---------- Synthetic (existing) ----------
# A: Moons
X_all, y_all = make_moons(n_samples=10000, noise=0.30, random_state=0)
idx = RNG.permutation(len(X_all))
tr, te = idx[:1000], idx[1000:2000]
save_npz(X_all[tr], y_all[tr], X_all[te], y_all[te], "synthetic_A_moons_1k_each")
save_npz(X_all, y_all, X_all, y_all, "synthetic_A_moons_full10k")

# B: Circles
Xb, yb = make_circles(n_samples=2000, noise=0.10, factor=0.5, random_state=1)
save_npz(Xb[:1000], yb[:1000], Xb[1000:2000], yb[1000:2000], "synthetic_B_circles_1k_each")

# C: Blobs
Xc, yc = make_blobs(n_samples=2000, centers=[(-1,0),(1,0)], cluster_std=[1.1,1.1], random_state=2)
save_npz(Xc[:1000], yc[:1000], Xc[1000:2000], yc[1000:2000], "synthetic_C_blobs_1k_each")

# =========================
# ---------- Paper Synthetic A/B/C (ADDED) ----------

def _split_balanced(X0, X1, n_train_each=1000, n_test_each=1000, rng=RNG):
    """X0, X1: (N0, d) ve (N1, d). Her sınıftan n_train_each + n_test_each ayırır."""
    assert len(X0) >= n_train_each + n_test_each and len(X1) >= n_train_each + n_test_each
    i0 = rng.permutation(len(X0))
    i1 = rng.permutation(len(X1))
    X0_tr = X0[i0[:n_train_each]]; X0_te = X0[i0[n_train_each:n_train_each+n_test_each]]
    X1_tr = X1[i1[:n_train_each]]; X1_te = X1[i1[n_train_each:n_train_each+n_test_each]]
    Xtr = np.vstack([X0_tr, X1_tr]); ytr = np.hstack([np.zeros(len(X0_tr), dtype=np.int64), np.ones(len(X1_tr), dtype=np.int64)])
    Xte = np.vstack([X0_te, X1_te]); yte = np.hstack([np.zeros(len(X0_te), dtype=np.int64), np.ones(len(X1_te), dtype=np.int64)])
    # karıştır
    p_tr = rng.permutation(len(Xtr)); p_te = rng.permutation(len(Xte))
    return Xtr[p_tr], ytr[p_tr], Xte[p_te], yte[p_te]

# --- Paper Synthetic A: "Sinusoidal bands" (2D) ---
# İki sınıf: y = sin(x) + offset ve y = sin(x) - offset çevresinde gürültülü şeritler.
def make_sin_bands(n_each=4000, x_range=(-np.pi, np.pi), offset=0.9, noise=0.15, rng=RNG):
    xs0 = rng.uniform(x_range[0], x_range[1], size=n_each)
    xs1 = rng.uniform(x_range[0], x_range[1], size=n_each)
    y0 = np.sin(xs0) - offset + rng.normal(0, noise, size=n_each)
    y1 = np.sin(xs1) + offset + rng.normal(0, noise, size=n_each)
    X0 = np.stack([xs0, y0], axis=1)
    X1 = np.stack([xs1, y1], axis=1)
    return X0, X1

X0, X1 = make_sin_bands(n_each=4000, offset=0.9, noise=0.12)
Xa_tr, ya_tr, Xa_te, ya_te = _split_balanced(X0, X1, 1000, 1000)
save_npz(Xa_tr, ya_tr, Xa_te, ya_te, "paper_syn_A_sin_bands_1k_each")

# --- Paper Synthetic B: "Two spirals" (2D, topolojik dolaşıklık) ---
def make_two_spirals(n_each=4000, noise=0.5, turns=3, rng=RNG):
    # klasik çift-spiral: r = a*theta
    theta = rng.uniform(0.0, turns * np.pi, size=n_each)
    a = 0.5
    r = a * theta
    x0 = r * np.cos(theta) + rng.normal(0, noise, size=n_each)
    y0 = r * np.sin(theta) + rng.normal(0, noise, size=n_each)
    # ikinci spiral π faz kaydır
    theta2 = theta + np.pi
    r2 = a * theta2
    x1 = r2 * np.cos(theta2) + rng.normal(0, noise, size=n_each)
    y1 = r2 * np.sin(theta2) + rng.normal(0, noise, size=n_each)
    X0 = np.stack([x0, y0], axis=1)
    X1 = np.stack([x1, y1], axis=1)
    return X0, X1

X0, X1 = make_two_spirals(n_each=4000, noise=0.35, turns=3.5)
Xb_tr, yb_tr, Xb_te, yb_te = _split_balanced(X0, X1, 1000, 1000)
save_npz(Xb_tr, yb_tr, Xb_te, yb_te, "paper_syn_B_two_spirals_1k_each")

# --- Paper Synthetic C: "Sine surface vs plane" (3D, sinüs yüzeyi ile düzlemin kesişmesi) ---
# Sınıf-0: z ≈ 0 düzlemi çevresinde gürültülü nokta bulutu
# Sınıf-1: z ≈ sin(x) + 0.5*sin(y) yüzeyi çevresinde gürültülü nokta bulutu
def make_sine_surface_vs_plane(n_each=6000, xy_range=(-2*np.pi, 2*np.pi), noise_xy=0.05, noise_z=0.12, rng=RNG):
    # Plane (class 0)
    x0 = rng.uniform(xy_range[0], xy_range[1], size=n_each)
    y0 = rng.uniform(xy_range[0], xy_range[1], size=n_each)
    z0 = rng.normal(0, noise_z, size=n_each)  # z ~ 0
    X0 = np.stack([x0 + rng.normal(0, noise_xy, size=n_each),
                   y0 + rng.normal(0, noise_xy, size=n_each),
                   z0], axis=1)
    # Sine surface (class 1)
    x1 = rng.uniform(xy_range[0], xy_range[1], size=n_each)
    y1 = rng.uniform(xy_range[0], xy_range[1], size=n_each)
    z1_mean = np.sin(x1) + 0.5*np.sin(y1)
    z1 = z1_mean + rng.normal(0, noise_z, size=n_each)
    X1 = np.stack([x1 + rng.normal(0, noise_xy, size=n_each),
                   y1 + rng.normal(0, noise_xy, size=n_each),
                   z1], axis=1)
    return X0, X1

X0, X1 = make_sine_surface_vs_plane(n_each=6000, xy_range=(-3.0, 3.0), noise_xy=0.03, noise_z=0.08)
Xc_tr, yc_tr, Xc_te, yc_te = _split_balanced(X0, X1, 1000, 1000)
save_npz(Xc_tr, yc_tr, Xc_te, yc_te, "paper_syn_C_sine_surface_vs_plane_1k_each")

# =========================
# ---------- Helpers ----------
def subset_digits_flat(x, y, pos, neg):
    sel = np.where((y==pos)|(y==neg))[0]
    X = x[sel].astype('float32')/255.0
    yb = (y[sel]==pos).astype('int64')
    X = X.reshape((X.shape[0], -1))
    return X, yb

def _tfds_to_numpy(ds, take=None):
    Xs, ys = [], []
    for i, ex in enumerate(tfds.as_numpy(ds)):
        if (take is not None) and (i>=take): break
        Xs.append(ex['image'])
        ys.append(ex['label'])
    X = np.stack(Xs).astype('float32')/255.0
    y = np.array(ys).astype('int64')
    return X, y

def subset_pair(X, y, a, b):
    m = (y==a)|(y==b)
    X = X[m]; y = y[m]
    yb = (y==a).astype('int64')
    return X.reshape((X.shape[0], -1)), yb

# =========================
# ---------- MNIST (çoklu ikili) ----------
(Xtr, ytr), (Xte, yte) = mnist.load_data()
mnist_pairs = [
    (1,7,'mnist_1v7'),
    (6,8,'mnist_6v8'),
    (0,8,'mnist_0v8'),
    (2,7,'mnist_2v7'),
    (3,5,'mnist_3v5'),
    (4,9,'mnist_4v9'),
]
for a,b,name in mnist_pairs:
    Xtr2, ytr2 = subset_digits_flat(Xtr, ytr, a, b)
    Xte2, yte2 = subset_digits_flat(Xte, yte, a, b)
    save_npz(Xtr2, ytr2, Xte2, yte2, name)

# ---------- Fashion-MNIST (çoklu ikili) ----------
(Xtr, ytr), (Xte, yte) = fashion_mnist.load_data()
fmnist_pairs = [
    (5,9,'fmnist_sandal_v_boot'),
    (6,4,'fmnist_shirt_v_coat'),
    (0,6,'fmnist_tshirt_v_shirt'),
]
for a,b,name in fmnist_pairs:
    Xtr2, ytr2 = subset_digits_flat(Xtr, ytr, a, b)
    Xte2, yte2 = subset_digits_flat(Xte, yte, a, b)
    save_npz(Xtr2, ytr2, Xte2, yte2, name)

# ---------- CIFAR-10 (iki ikili) ----------
# labels: 0=airplane,1=automobile,2=bird,3=cat,4=deer,5=dog,6=frog,7=horse,8=ship,9=truck
(Xtr, ytr), (Xte, yte) = cifar10.load_data()
ytr = ytr.ravel().astype('int64'); yte = yte.ravel().astype('int64')
# cat(3) vs dog(5)
Xtr2, ytr2 = subset_pair(Xtr, ytr, 3, 5)
Xte2, yte2 = subset_pair(Xte, yte, 3, 5)
save_npz(Xtr2, ytr2, Xte2, yte2, "cifar10_cat_v_dog")
# automobile(1) vs truck(9)
Xtr2, ytr2 = subset_pair(Xtr, ytr, 1, 9)
Xte2, yte2 = subset_pair(Xte, yte, 1, 9)
save_npz(Xtr2, ytr2, Xte2, yte2, "cifar10_car_v_truck")

# ---------- SVHN (iki ikili) ----------
svhn_train = tfds.load('svhn_cropped', split='train', shuffle_files=False, try_gcs=True)
svhn_test  = tfds.load('svhn_cropped', split='test',  shuffle_files=False, try_gcs=True)
Xtr, ytr = _tfds_to_numpy(svhn_train)  # büyükse ileride örnekleme ekleyebiliriz
Xte, yte = _tfds_to_numpy(svhn_test)
# 1 vs 7
Xtr2, ytr2 = subset_pair(Xtr, ytr, 1, 7)
Xte2, yte2 = subset_pair(Xte, yte, 1, 7)
save_npz(Xtr2, ytr2, Xte2, yte2, "svhn_1v7")
# 3 vs 8
Xtr2, ytr2 = subset_pair(Xtr, ytr, 3, 8)
Xte2, yte2 = subset_pair(Xte, yte, 3, 8)
save_npz(Xtr2, ytr2, Xte2, yte2, "svhn_3v8")


In [ ]:
#@title Görselleştirme: data/*.npz içindeki TÜM datasetler için otomatik çizim
import os, math, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

DATA_DIR = Path("data")
MAX_PLOT_POINTS = 3000     # scatter için üst sınır (performans/okunurluk)
IMG_PER_CLASS = 8          # pano başına sınıf başına örnek
SEED = 123
RNG = np.random.default_rng(SEED)

# Yalnızca belirli datasetleri çizmek istersen isim (npz adı) ekle:
INCLUDE_ONLY = None  # örn: ["paper_syn_A_sin_bands_1k_each", "mnist_1v7"]

def _class_counts(y):
    vals, cnts = np.unique(y, return_counts=True)
    return dict(zip(vals.tolist(), cnts.tolist()))

def _subsample(X, y, max_n=MAX_PLOT_POINTS):
    n = len(X)
    if n <= max_n: 
        return X, y
    idx = RNG.permutation(n)[:max_n]
    return X[idx], y[idx]

def _grid_dims(n_panels):
    cols = min(8, n_panels)
    rows = math.ceil(n_panels / cols)
    return rows, cols

def _guess_image_shape(d):
    # Yaygın düzleştirilmiş görüntü boyutları için heuristik:
    if d == 28*28:          # MNIST / fMNIST
        return (28, 28, 1)
    if d == 32*32*3:        # CIFAR10 / SVHN
        return (32, 32, 3)
    if d == 32*32:          # muhtemel 32x32 grayscale
        return (32, 32, 1)
    return None

def _plot_images_grid(X, y, img_shape, title="", img_per_class=IMG_PER_CLASS):
    classes = sorted(np.unique(y).tolist())
    n_panels = len(classes) * img_per_class
    rows, cols = _grid_dims(n_panels)
    plt.figure(figsize=(cols*1.4, rows*1.4))
    k = 1
    for c in classes:
        idx = np.where(y==c)[0]
        if len(idx)==0: 
            continue
        take = min(img_per_class, len(idx))
        pick = RNG.choice(idx, size=take, replace=False)
        for i in pick:
            plt.subplot(rows, cols, k)
            img = X[i].reshape(img_shape)
            if img_shape[2] == 1:
                plt.imshow(img.squeeze(), cmap="gray")
            else:
                plt.imshow(img)
            plt.axis("off")
            plt.title(f"y={c}", fontsize=8)
            k += 1
    plt.suptitle(title, fontsize=12)
    plt.tight_layout()
    plt.show()

def _scatter2d(X, y, title=""):
    Xs, ys = _subsample(X, y)
    plt.figure(figsize=(5.2, 4.6))
    plt.scatter(Xs[:,0], Xs[:,1], c=ys, s=8, alpha=0.8)
    plt.title(title)
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

def _scatter3d(X, y, title=""):
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    Xs, ys = _subsample(X, y)
    fig = plt.figure(figsize=(6, 5.2))
    ax = fig.add_subplot(111, projection="3d")
    p = ax.scatter(Xs[:,0], Xs[:,1], Xs[:,2], c=ys, s=8, alpha=0.85)
    ax.set_title(title)
    ax.set_xlabel("x1"); ax.set_ylabel("x2"); ax.set_zlabel("x3")
    plt.tight_layout(); plt.show()

def _pca2(X, n_components=2):
    from sklearn.decomposition import PCA
    return PCA(n_components=n_components).fit_transform(X)

def visualize_npz_dataset(npz_path: Path):
    name = npz_path.stem
    data = np.load(npz_path)
    Xtr, ytr = data["X_train"], data["y_train"]
    d = Xtr.shape[1] if Xtr.ndim == 2 else (np.prod(Xtr.shape[1:]) if Xtr.ndim>2 else 1)
    if Xtr.ndim > 2:  # güvenli tarafta ol
        Xtr = Xtr.reshape((Xtr.shape[0], -1))
        d = Xtr.shape[1]

    # Özet bilgi
    print(f"\n=== {name} ===")
    print(f"train: {Xtr.shape}, classes: {sorted(np.unique(ytr).tolist())}, counts: {_class_counts(ytr)}")
    
    # Görsel karar ağacı
    img_shape = _guess_image_shape(d)
    if d == 2:
        _scatter2d(Xtr, ytr, title=f"{name} (2D)")
    elif d == 3:
        _scatter3d(Xtr, ytr, title=f"{name} (3D)")
    elif img_shape is not None:
        _plot_images_grid(Xtr, ytr, img_shape, title=f"{name} (images)")
    else:
        # Genel durum: PCA(2D) + scatter (tek subsample çağrısı!)
        Xs, ys = _subsample(Xtr, ytr)
        Z = _pca2(Xs, n_components=2)
        plt.figure(figsize=(5.2,4.6))
        plt.scatter(Z[:,0], Z[:,1], c=ys, s=8, alpha=0.8)
        plt.title(f"{name} (PCA→2D)")
        plt.xlabel("PC1"); plt.ylabel("PC2")
        plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()


# ==== Ana akış: tüm .npz dosyalarını çiz ====
npz_files = sorted([p for p in DATA_DIR.glob("*.npz")])
if INCLUDE_ONLY is not None:
    npz_files = [p for p in npz_files if p.stem in INCLUDE_ONLY]

if not npz_files:
    print("data/ içinde .npz bulunamadı. Önce veri hücresini çalıştır.")
else:
    for p in npz_files:
        try:
            visualize_npz_dataset(p)
        except Exception as e:
            print(f"[WARN] {p.name} çizilemedi: {e}")


In [ ]:
#@title MLP factory (ReLU, RMSprop 1e-3, BCE) — NPZ'den otomatik input_dim
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_mlp(input_dim, width='narrow', depth=5,
              hidden_activation='relu',
              out_activation='sigmoid',
              optimizer='rmsprop',
              learning_rate=1e-3):
    """
    width: ['narrow','wide','bottleneck']
    depth: narrow/wide için katman sayısı; bottleneck için {5,11}
    """
    assert width in ['narrow','wide','bottleneck'], "width ∈ {'narrow','wide','bottleneck'} olmalı"

    inputs = keras.Input(shape=(input_dim,))
    x = inputs

    if width == 'bottleneck':
        if depth == 5:
            units = [50,50,25,50,50]
        elif depth == 11:
            units = [50,50,50,50,50,25,50,50,50,50,50]
        else:
            raise ValueError("bottleneck için depth {5,11} olmalı")
        for u in units:
            x = layers.Dense(u, activation=hidden_activation)(x)
    else:
        hidden_units = 25 if width=='narrow' else 50
        for _ in range(depth):
            x = layers.Dense(hidden_units, activation=hidden_activation)(x)

    outputs = layers.Dense(1, activation=out_activation)(x)
    model = keras.Model(inputs, outputs)

    # optimizer seçimi
    opt = optimizer.lower()
    if opt == 'rmsprop':
        opt_obj = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif opt == 'adam':
        opt_obj = keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'sgd':
        opt_obj = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        raise ValueError("optimizer ∈ {'rmsprop','adam','sgd'}")

    model.compile(optimizer=opt_obj, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def _npz_input_dim(npz_path):
    """
    data/<name>.npz dosyasını okuyup X_train'in sütun sayısını döndürür.
    (Sentetik 2D/3D setler veya önceden flatten edilmiş görüntülerle uyumlu.)
    """
    with np.load(npz_path) as data:
        Xtr = data['X_train']
    Xtr = Xtr.reshape((Xtr.shape[0], -1))
    return Xtr.shape[1]

def build_mlp_for_npz(npz_path, width='narrow', depth=5,
                      hidden_activation='relu',
                      out_activation='sigmoid',
                      optimizer='rmsprop',
                      learning_rate=1e-3):
    input_dim = _npz_input_dim(npz_path)
    return build_mlp(input_dim,
                     width=width, depth=depth,
                     hidden_activation=hidden_activation,
                     out_activation=out_activation,
                     optimizer=optimizer,
                     learning_rate=learning_rate)


In [ ]:
# --- kısa smoke test (yorumdan çıkarıp herhangi bir NPZ ile deneyebilirsin) ---
m = build_mlp_for_npz("data/paper_syn_A_sin_bands_1k_each.npz", width='bottleneck', depth=5)
m.summary()


In [ ]:
#@title FIX+EXT: Keras 3 uyumlu embedding çıkarımı + standardizasyon + run_cycle
from pathlib import Path
import json, glob
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

EMB_DIR = Path("embeddings"); EMB_DIR.mkdir(exist_ok=True, parents=True)
CKPT_DIR = Path("checkpoints"); CKPT_DIR.mkdir(exist_ok=True, parents=True)
LOGS_DIR = Path("logs"); LOGS_DIR.mkdir(exist_ok=True, parents=True)

def _standardize_train_test(Xtr, Xte):
    mu = Xtr.mean(axis=0, keepdims=True)
    sd = Xtr.std(axis=0, keepdims=True) + 1e-8
    return (Xtr - mu)/sd, (Xte - mu)/sd, mu.astype(np.float32), sd.astype(np.float32)

def _ensure_float32(*arrays):
    return [a.astype(np.float32, copy=False) for a in arrays]

def run_one(dataset_npz, width, depth, epochs=50, batch_size=128, seed=0, standardize=True, save_meta=True):
    """
    dataset_npz: Path to .npz with keys (X_train,y_train,X_test,y_test)
    Returns: (test_acc: float, base: str, layer_names: list[str])
    """
    d = np.load(dataset_npz)
    Xtr, ytr, Xte, yte = d['X_train'], d['y_train'], d['X_test'], d['y_test']
    # Keras beklentisi için dtype ve şekil
    Xtr, Xte = _ensure_float32(Xtr, Xte)
    ytr = ytr.astype(np.int32, copy=False)
    yte = yte.astype(np.int32, copy=False)

    if standardize:
        Xtr, Xte, mu, sd = _standardize_train_test(Xtr, Xte)
    else:
        mu = np.zeros((1, Xtr.shape[1]), dtype=np.float32)
        sd = np.ones((1, Xtr.shape[1]), dtype=np.float32)

    input_dim = Xtr.shape[1]
    tf.keras.utils.set_random_seed(seed)

    # build_mlp kullanıcı hücresinde tanımlı (binary çıktı varsayımı)
    model = build_mlp(input_dim, width=width, depth=depth)
    _ = model.fit(Xtr, ytr, validation_data=(Xte, yte), epochs=epochs, batch_size=batch_size, verbose=0)
    te_loss, te_acc = model.evaluate(Xte, yte, verbose=0)

    # --- Keras 3 uyumlu ara katman çıktıları ---
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    layer_names = [lyr.name for lyr in dense_layers]
    inter_model = keras.Model(inputs=model.input, outputs=[lyr.output for lyr in dense_layers])
    emb_list = inter_model.predict(Xte, batch_size=1024, verbose=0)  # list[np.ndarray]

    # Kaydetme
    base = Path(dataset_npz).stem + f"__w{width}_d{depth}_seed{seed}"
    out_dir = (EMB_DIR / base); out_dir.mkdir(parents=True, exist_ok=True)
    for name, E in zip(layer_names, emb_list):
        np.save(out_dir / f"{name}.npy", E)
    np.save(out_dir / "y_test.npy", yte)
    np.save(out_dir / "X_test.npy", Xte)
    np.save(out_dir / "mu.npy", mu)  # standardizasyon istatistikleri
    np.save(out_dir / "sd.npy", sd)

    # Model checkpoint
    model.save(CKPT_DIR / f"{base}.keras")

    # Meta/Log
    if save_meta:
        meta = {
            "dataset": str(dataset_npz),
            "width": int(width),
            "depth": int(depth),
            "epochs": int(epochs),
            "batch_size": int(batch_size),
            "seed": int(seed),
            "standardize": bool(standardize),
            "test_acc": float(te_acc),
            "layers": layer_names,
            "embedding_dir": str(out_dir),
            "checkpoint": str(CKPT_DIR / f"{base}.keras"),
        }
        with open(LOGS_DIR / f"{base}.json", "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)

    return float(te_acc), base, layer_names

# --------------------------------------------------------------------
# Yardımcı: dataset bulucu (pattern ile)
def find_datasets(patterns=("data/*.npz",), include_only=None, exclude_substr=None):
    """
    patterns: glob desenleri tuple/list
    include_only: None veya substring listesi (sadece bunları içerenleri al)
    exclude_substr: None veya substring listesi (bunları içerenleri hariç tut)
    """
    files = []
    for p in patterns:
        files.extend(glob.glob(p))
    files = sorted(set(files))
    if include_only:
        files = [f for f in files if any(s in f for s in include_only)]
    if exclude_substr:
        files = [f for f in files if not any(s in f for s in exclude_substr)]
    return files

# --------------------------------------------------------------------
# Toplu çalıştırma (cycle)
def run_cycle(widths=(64,128), depths=(3,5), seeds=(0,), epochs=50, batch_size=128,
              patterns=("data/*.npz",),
              include_only=None,  # örn: ["paper_syn_", "mnist_"]
              exclude_substr=None,  # örn: ["full10k"] gibi ağır dosyaları dışla
              standardize=True):
    """
    Örnek:
    run_cycle(widths=(64,), depths=(5,), seeds=(0,1,2),
              include_only=["paper_syn_", "mnist_", "cifar10_car_v_truck"])
    """
    ds_list = find_datasets(patterns, include_only=include_only, exclude_substr=exclude_substr)
    results = []
    for ds in ds_list:
        for w in widths:
            for d in depths:
                for s in seeds:
                    acc, base, layers = run_one(ds, w, d, epochs=epochs, batch_size=batch_size,
                                                seed=s, standardize=standardize, save_meta=True)
                    results.append((ds, w, d, s, acc, base))
                    print(f"[OK] {base}  acc={acc:.4f}")
    # CSV dök
    import csv
    csv_path = LOGS_DIR / "run_cycle_results.csv"
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        wtr = csv.writer(f)
        wtr.writerow(["dataset","width","depth","seed","test_acc","run_id"])
        for (ds, w, d, s, acc, base) in results:
            wtr.writerow([ds, w, d, s, f"{acc:.6f}", base])
    print("Saved:", csv_path)
    return results


In [ ]:
# --- PAPER SYNTHETICS (A/B/C) ---
run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=20, batch_size=256, include_only=["paper_syn_A_sin_bands_1k_each"])

run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=20, batch_size=256, include_only=["paper_syn_B_two_spirals_1k_each"])

run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=20, batch_size=256, include_only=["paper_syn_C_sine_surface_vs_plane_1k_each"])


In [ ]:
# --- MNIST (çoklu ikili) ---
run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=15, batch_size=256, include_only=["mnist_1v7"])

run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=15, batch_size=256, include_only=["mnist_6v8"])

run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=15, batch_size=256, include_only=["mnist_0v8","mnist_2v7","mnist_3v5","mnist_4v9"])


In [ ]:
# --- Fashion-MNIST (çoklu ikili) ---
run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=15, batch_size=256, include_only=["fmnist_sandal_v_boot"])

run_cycle(widths=(64,), depths=(3,), seeds=(0,),
          epochs=15, batch_size=256, include_only=["fmnist_shirt_v_coat","fmnist_tshirt_v_shirt"])


In [ ]:
# --- CIFAR-10 (iki ikili) ---
run_cycle(widths=(128,), depths=(5,), seeds=(0,),
          epochs=20, batch_size=256, include_only=["cifar10_car_v_truck"])

run_cycle(widths=(128,), depths=(5,), seeds=(0,),
          epochs=20, batch_size=256, include_only=["cifar10_cat_v_dog"])


In [ ]:
# --- SVHN (iki ikili) ---
# Not: SVHN büyük; yine de tek seed + 20 epoch kullanıyoruz.
run_cycle(widths=(128,), depths=(5,), seeds=(0,),
          epochs=20, batch_size=512, include_only=["svhn_1v7"])

run_cycle(widths=(128,), depths=(5,), seeds=(0,),
          epochs=20, batch_size=512, include_only=["svhn_3v8"])


In [ ]:
import pandas as pd, json, glob, numpy as np

# 1) Son koşular tablosu
df = pd.read_csv("logs/run_cycle_results.csv")
display(df.tail(12))

# 2) ρ_k – Accuracy (varsa rho'ları birleştir)
# Elinde rho'lar başka CSV'de ise oradan merge et; şimdilik sadece test_acc dağılımı:
scatter_rho_vs_acc(df, by="dataset")  # rho kolonu mevcut dosyanda varsa çizilecek

# 3) Örnek bir koşunun katman trendleri
# Son koşunun run_id’sinden embedding klasörü:
run_id = df.iloc[-1]["run_id"]
emb_dir = Path("embeddings")/run_id
# Bu kısmı senin Ric_l ve g_l üretim fonksiyonlarına bağlı; örnek çağrılar:
# Ric_list = compute_total_forman_ricci_over_layers(emb_dir, k=15)
# g_list   = compute_total_geodesic_over_layers(emb_dir, k=15)
# plot_trends_over_layers(Ric_list, g_list)


In [ ]:
#@title Ricci metrics (fast): ignore last layer, sampling, giant component [UPDATED]
import numpy as np
from sklearn.neighbors import kneighbors_graph
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path, connected_components
from scipy.stats import pearsonr
from pathlib import Path
import re

def _bin_adjacency(A):
    B = (A > 0).astype(np.int8).tocsr()
    return B.maximum(B.T)

def _largest_component_mask(B):
    ncomp, labels = connected_components(csgraph=B, directed=False, return_labels=True)
    if ncomp == 1:
        return np.ones(B.shape[0], dtype=bool), 1.0
    counts = np.bincount(labels)
    giant = np.argmax(counts)
    mask = (labels == giant)
    ratio = counts[giant] / B.shape[0]
    return mask, ratio

def _forman_ricci_total(B):
    deg = np.ravel(B.sum(axis=1)).astype(np.int64)
    rows, cols = B.nonzero()
    mask = rows < cols
    ri = rows[mask]; rj = cols[mask]
    return int(np.sum(4 - deg[ri] - deg[rj]))

def layerwise_global_metrics_fast(
    emb_dir: Path,
    layer_names,
    k_values,
    ignore_last_layer: bool = True,
    sample_size: int | None = None,
    min_component_ratio: float = 0.99,
    rng_seed: int = 0,
    return_connectivity: bool = True,
):
    """
    Her k için:
      - Ric_l toplamı (Forman) ve g_l toplam geodezikten r = corr(eta_l, Ric_l) ve z hesaplar.
      - İsteğe bağlı: connectivity ısı haritası için (k,l) bazlı giant component ratio döndürür.
    """
    use_layers = layer_names[:-1] if ignore_last_layer and len(layer_names) > 1 else layer_names[:]
    if len(use_layers) < 2:
        return [(k, np.nan, np.nan, len(use_layers), False) for k in k_values], None

    y = np.load(emb_dir / "y_test.npy")
    n_total = y.shape[0]
    rng = np.random.default_rng(rng_seed)

    # örnekleme (node subset)
    if sample_size is not None and sample_size < n_total:
        idx = np.sort(rng.choice(n_total, size=sample_size, replace=False))
    else:
        idx = np.arange(n_total)

    results = []
    conn_map = {}  # k -> [ratio per layer]
    for k in k_values:
        Ric, gvals = [], []
        valid = True
        ratios = []

        for lname in use_layers:
            Z = np.load(emb_dir / f"{lname}.npy")[idx]  # (m, d_l)
            A = kneighbors_graph(Z, n_neighbors=k, mode='distance', include_self=False, n_jobs=-1)
            B = _bin_adjacency(A)

            # dev bileşen toleransı
            mask, ratio = _largest_component_mask(B)
            ratios.append(float(ratio))
            if ratio < min_component_ratio:
                valid = False
                # devam etmeden bu k'yı invalid işaretliyoruz
                break

            if not mask.all():
                sel = np.where(mask)[0]
                B = B[sel][:, sel]

            # Ricci toplamı (Forman)
            ric_total = _forman_ricci_total(B)
            Ric.append(float(ric_total))

            # geodezik toplamı (unweighted BFS)
            D = shortest_path(B, directed=False, unweighted=True)
            if np.isinf(D).any():
                valid = False
                break
            gvals.append(float(np.sum(D)))

        if return_connectivity:
            conn_map[k] = ratios if len(ratios)==len(use_layers) else [np.nan]*len(use_layers)

        if not valid:
            results.append((k, np.nan, np.nan, len(use_layers), False))
            continue

        Ric = np.asarray(Ric, dtype=float)
        gvals = np.asarray(gvals, dtype=float)
        eta = gvals[1:] - gvals[:-1]

        # sabit vektörler NaN üretmesin diye kontrol
        if np.allclose(eta, eta[0]) or np.allclose(Ric[:-1], Ric[:-1][0]):
            results.append((k, np.nan, np.nan, len(use_layers), False))
            continue

        r, _ = pearsonr(eta, Ric[:-1])
        # Fisher z: küçük L'de dikkat
        denom = max(len(use_layers) - 4, 1)
        z = np.arctanh(np.clip(r, -0.999999, 0.999999)) / np.sqrt(denom)
        results.append((k, float(r), float(z), len(use_layers), True))

    return results, conn_map if return_connectivity else None


def _infer_dataset_mode_from_runid(run_id: str) -> str:
    """
    run_id'dan sentetik/real ipucu çıkar.
    - 'paper_syn', 'synthetic_' gibi önekler → 'synthetic'
    - aksi → 'real'
    """
    rid = run_id.lower()
    if re.search(r"(paper_syn|synthetic_|syn_|_syn_|moons|spiral|circle|blobs)", rid):
        return "synthetic"
    return "real"


def auto_k_sweep_fast(
    run_id,
    layer_names,
    mode: str | None = None,
    sample_size=None,
    min_component_ratio=0.99,
    rng_seed=0,
    return_connectivity: bool = True,
):
    """
    mode=None ise run_id'a bakarak 'synthetic'/'real' belirlenir.
    - synthetic: daha küçük k'ler de denenir (küçük N ve “daha kıvrımlı” graf yapıları için).
    - real: daha geniş k ızgarası.
    """
    if mode is None:
        mode = _infer_dataset_mode_from_runid(run_id)

    if mode == 'synthetic':
        k_values = [4,5,6,7,9,10,12,15,18,20,30,50,90]
    else:
        k_values = [10,20,30,50,90,120,150,200,250,350]

    emb_dir = Path("embeddings") / run_id
    res, conn_map = layerwise_global_metrics_fast(
        emb_dir, layer_names, k_values,
        ignore_last_layer=True,            # <-- kritik
        sample_size=sample_size,           # örn. 800
        min_component_ratio=min_component_ratio,
        rng_seed=rng_seed,
        return_connectivity=return_connectivity,
    )
    # en negatif r en başa
    good = [r for r in res if r[-1] and not np.isnan(r[1])]
    best = sorted(good, key=lambda x: x[1])[0] if good else None
    return res, best, k_values, conn_map


In [ ]:
# id1: run_id, ln1: layer_names listesi
res, best, ks, conn = auto_k_sweep_fast(id1, ln1, mode='synthetic', sample_size=None, rng_seed=0)

print("k, rho, z, L, connected")
for t in res:
    print(t)
print("BEST:", best)

# (opsiyonel) bağlı bileşen ısı haritası: conn[k] -> her katman için ratio
# connectivity_heatmap fonksiyonunu daha önce verdiğim hücreden çağırabilirsin:
connectivity_heatmap(conn)


In [ ]:
#@title ProgressBoard yardımcıları (reset-safe, grid-aware)
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Iterable
import time, pandas as pd, numpy as np
from IPython.display import clear_output, display
import math
from itertools import product

@dataclass
class ProgressBoard:
    columns: List[str]
    title: str = "Progress"
    rows: List[Dict[str, Any]] = field(default_factory=list)
    total: Optional[int] = None
    t0: float = field(default_factory=time.time)
    auto_cols: bool = True            # yeni alanları kolonlara otomatik ekle
    sort_pref: List[str] = field(default_factory=lambda: ["dataset","mode","width","depth","k","seed","elapsed_s"])
    _last_render: float = field(default_factory=lambda: 0.0)
    _render_min_interval: float = 0.2 # saniye; çok hızlı add() spam’ında ekranı yormasın

    # --- API ---
    def set_total(self, total: int):
        self.total = int(total)
        self.render(force=True)

    def bump_total(self, delta: int):
        self.total = (self.total or 0) + int(delta)
        self.render(force=True)

    def set_total_from_grid(self,
                            datasets: Iterable[str],
                            widths: Iterable[Any],
                            depths: Iterable[Any],
                            seeds: Iterable[int],
                            ks: Optional[Iterable[int]] = None,
                            repeats: int = 1):
        """
        grid toplam adımı hesaplar:
          |datasets| × |widths| × |depths| × |seeds| × (|ks| veya 1) × repeats
        """
        n = len(list(datasets)) * len(list(widths)) * len(list(depths)) * len(list(seeds))
        n *= (len(list(ks)) if ks is not None else 1)
        n *= int(repeats)
        self.set_total(n)

    def add(self, **kwargs):
        """
        Örn:
        add(dataset="paper_syn_A_sin_bands_1k_each", mode="real", width="narrow", depth=7,
            k=25, seed=0, test_acc=0.912, rho=-0.63, z=-0.71, connected_all_layers=True,
            elapsed_s=123.4, run_id="...") 
        """
        self.rows.append(kwargs)
        self.render()

    # --- İç ---
    def _compute_eta(self, done: int, elapsed: float):
        if not self.total or done == 0: 
            return None
        rate = done / max(elapsed, 1e-9)
        remain = self.total - done
        eta_s = remain / max(rate, 1e-9)
        return eta_s

    def render(self, force: bool=False):
        now = time.time()
        if not force and (now - self._last_render) < self._render_min_interval:
            return
        self._last_render = now

        clear_output(wait=True)

        # DataFrame’e dök
        df = pd.DataFrame(self.rows)

        # kolonları tamamla / yeni metrikleri ekle
        if self.auto_cols and len(df):
            unseen = [c for c in df.columns if c not in self.columns]
            if unseen:
                self.columns += unseen

        # eksik kolonları doldur
        for c in self.columns:
            if c not in df.columns:
                df[c] = np.nan

        # sıralama
        sort_cols = [c for c in self.sort_pref if c in df.columns]
        if sort_cols:
            with pd.option_context('mode.use_inf_as_na', True):
                df = df.sort_values(by=sort_cols, na_position='last', kind="mergesort")

        # sadece tanımlı kolonları göster
        df = df[self.columns]

        # başlık
        print(f"=== {self.title} ===")
        display(df)

        # durum satırı
        done = len(self.rows)
        elapsed = time.time() - self.t0
        if self.total:
            pct = 100.0 * done / max(self.total, 1)
            eta_s = self._compute_eta(done, elapsed)
            eta_str = f" | ETA ~{_fmt_duration(eta_s)}" if eta_s is not None and eta_s < 9e6 else ""
            print(f"[{self.title}] {done}/{self.total} = {pct:.1f}% | elapsed {_fmt_duration(elapsed)}{eta_str}")
        else:
            print(f"[{self.title}] steps={done} | elapsed {_fmt_duration(elapsed)}")

        # küçük özet: dataset bazlı tamamlanan sayılar
        if len(df) and "dataset" in df.columns:
            cnt = df["dataset"].value_counts().sort_index()
            with pd.option_context('display.max_rows', 200, 'display.width', 120):
                print("\nCompleted per dataset:")
                display(cnt.to_frame("done"))

def timed_step(fn, *args, **kwargs):
    t0 = time.time()
    out = fn(*args, **kwargs)
    return out, round(time.time()-t0, 1)

def _fmt_duration(sec: float) -> str:
    if sec is None: return "?"
    sec = float(sec)
    if sec < 60: return f"{sec:.1f}s"
    m, s = divmod(int(round(sec)), 60)
    if m < 60: return f"{m}m{s:02d}s"
    h, m = divmod(m, 60)
    return f"{h}h{m:02d}m"

print("ProgressBoard (grid-aware) loaded.")


In [ ]:
#@title Grid runner (FAST) + Canlı İlerleme Paneli  ✅ (paper_syn_* eklendi)
from pathlib import Path
import time, pandas as pd, numpy as np
from IPython.display import clear_output, display
from tqdm.auto import tqdm

# 1) Çalıştırılacak dataset listesi
DATASETS = [
  # ---- Paper synthetic A/B/C (NEW) ----
  ("data/paper_syn_A_sin_bands_1k_each.npz", "synthetic"),
  ("data/paper_syn_B_two_spirals_1k_each.npz", "synthetic"),
  ("data/paper_syn_C_sine_surface_vs_plane_1k_each.npz", "synthetic"),

  # ---- Existing synthetic ----
  ("data/synthetic_A_moons_1k_each.npz", "synthetic"),
  ("data/synthetic_B_circles_1k_each.npz", "synthetic"),
  ("data/synthetic_C_blobs_1k_each.npz", "synthetic"),

  # ---- MNIST/fMNIST (çeşitlendirilmiş) ----
  ("data/mnist_1v7.npz", "real"),
  ("data/mnist_6v8.npz", "real"),
  ("data/mnist_0v8.npz", "real"),
  ("data/mnist_2v7.npz", "real"),
  ("data/mnist_3v5.npz", "real"),
  ("data/mnist_4v9.npz", "real"),
  ("data/fmnist_sandal_v_boot.npz", "real"),
  ("data/fmnist_shirt_v_coat.npz", "real"),
  ("data/fmnist_tshirt_v_shirt.npz", "real"),

  # ---- CIFAR-10 ----
  ("data/cifar10_cat_v_dog.npz", "real"),
  ("data/cifar10_car_v_truck.npz", "real"),

  # ---- SVHN ----
  ("data/svhn_1v7.npz", "real"),
  ("data/svhn_3v8.npz", "real"),
]

WIDTHS = ["narrow","wide","bottleneck"]
DEPTHS = [5,11]

def _sample_size_for(mode, ds_name):
    # synthetic: tamamını kullan (k-NN yapısı küçük/orta ölçekli)
    if mode == 'synthetic':
        return None
    # gerçek veriler için hızlı Ricci: dataset'e göre örneklem büyüklüğü
    if ds_name.startswith("cifar10") or ds_name.startswith("svhn"):
        return 1200   # daha büyük, çünkü sınıflar daha kompleks
    else:
        return 800    # mnist/fmnist

summary_rows = []
total = len(DATASETS)*len(WIDTHS)*len(DEPTHS)
i = 0

start_global = time.time()
for ds, mode in DATASETS:
    ds_name = Path(ds).stem
    for w in WIDTHS:
        for d in DEPTHS:
            i += 1
            step_t0 = time.time()
            try:
                acc, run_id, layer_names = run_one(ds, w, d, epochs=60, seed=0)
                ss = _sample_size_for(mode, ds_name)
                res, best = auto_k_sweep_fast(run_id, layer_names, mode=mode,
                                              sample_size=ss, rng_seed=0)
                row = {
                    "dataset": ds_name, "mode": mode,
                    "width": w, "depth": d,
                    "test_acc": float(acc),
                    "best_k": (int(best[0]) if best else None),
                    "rho": (float(best[1]) if best else None) if best else None,
                    "z": (float(best[2]) if best else None) if best else None,
                    "L": (int(best[3]) if best else None) if best else None,
                    "connected_all_layers": (bool(best[-1]) if best else False) if best else False,
                    "run_id": run_id,
                    "elapsed_s": round(time.time()-step_t0, 1)
                }
                summary_rows.append(row)
                status = f"DONE {ds_name:>38} | {w:10s} d={d:>2} | acc={acc:.4f} | best={best} | step={row['elapsed_s']}s"
                print(status)
            except Exception as e:
                row = {
                    "dataset": ds_name, "mode": mode,
                    "width": w, "depth": d,
                    "error": str(e)
                }
                summary_rows.append(row)
                print(f"SKIP {ds_name} {w} d={d} -> {e}")

            # canlı çıktı tablosu
            clear_output(wait=True)
            df_live = pd.DataFrame(summary_rows)
            # sıralama ve gösterim
            order_cols = ["dataset","mode","width","depth","test_acc","best_k","rho","z","connected_all_layers","elapsed_s","run_id","error"]
            for c in order_cols:
                if c not in df_live.columns:
                    df_live[c] = np.nan
            df_live = df_live[order_cols].sort_values(by=["dataset","width","depth","elapsed_s"], na_position='last')
            display(df_live)
            done_pct = 100.0 * i / total
            print(f"[{i}/{total}] {done_pct:.1f}% | total elapsed: {round(time.time()-start_global,1)}s")

# final kaydet
df = pd.DataFrame(summary_rows).sort_values(by=["dataset","width","depth"], na_position='last')
csv_path = Path("ricci_summary.csv"); df.to_csv(csv_path, index=False)
print("Saved ->", csv_path)
df


In [ ]:
#@title 5-seed istikrar analizi (panel, self-contained) — **UPDATED with paper_syn A/B/C**
# Guard: ProgressBoard/timed_step tanımlı değilse burada tanımla
try:
    ProgressBoard; timed_step
except NameError:
    from dataclasses import dataclass, field
    from typing import List, Dict, Any, Optional
    import time, pandas as pd, numpy as np
    from IPython.display import clear_output, display
    @dataclass
    class ProgressBoard:
        columns: List[str]
        title: str = "Progress"
        rows: List[Dict[str, Any]] = field(default_factory=list)
        total: Optional[int] = None
        t0: float = field(default_factory=time.time)
        def add(self, **kwargs):
            self.rows.append(kwargs); self.render()
        def render(self):
            clear_output(wait=True)
            df = pd.DataFrame(self.rows)
            for c in self.columns:
                if c not in df.columns: df[c] = np.nan
            df = df[self.columns]
            sort_cols = [c for c in ["dataset","mode","width","depth","seed","elapsed_s"] if c in df.columns]
            if sort_cols: df = df.sort_values(by=sort_cols, na_position='last')
            display(df)
            done = len(self.rows)
            if self.total:
                pct = 100.0 * done / self.total
                elapsed = time.time() - self.t0
                print(f"[{self.title}] {done}/{self.total} = {pct:.1f}% | elapsed {elapsed:.1f}s")
            else:
                print(f"[{self.title}] steps={done} | elapsed {time.time()-self.t0:.1f}s")
    def timed_step(fn, *args, **kwargs):
        t0 = time.time(); out = fn(*args, **kwargs); return out, round(time.time()-t0, 1)

from pathlib import Path
import numpy as np, pandas as pd, time, re, os

# ------------------------------------------------------------------
# Hedefler: Eski 5 + Yeni 3 (paper_syn_*). İstersen AUTO_DISCOVERY'yi True yap.
AUTO_DISCOVERY = False  # True -> data/ içinden desenle toplayıp otomatik hedef listesi kurar.

if not AUTO_DISCOVERY:
    targets = [
      # --- mevcut hedefler ---
      ("data/synthetic_A_moons_1k_each.npz","synthetic","narrow",5),
      ("data/mnist_1v7.npz","real","wide",5),
      ("data/fmnist_shirt_v_coat.npz","real","wide",11),
      ("data/svhn_3v8.npz","real","wide",11),
      ("data/cifar10_cat_v_dog.npz","real","wide",11),

      # --- YENİ: makaledeki sentetikler (A/B/C) ---
      ("data/paper_syn_A_sin_bands_1k_each.npz","synthetic","narrow",5),
      ("data/paper_syn_B_two_spirals_1k_each.npz","synthetic","narrow",5),
      ("data/paper_syn_C_sine_surface_vs_plane_1k_each.npz","synthetic","narrow",5),
    ]
else:
    # Otomatik keşif: data/ klasöründen belirli desenleri al.
    # Not: çok fazla hedef birikebilir; MAX_TARGETS ile sınırla.
    data_dir = Path("data")
    npzs = sorted(map(str, data_dir.glob("*.npz")))
    MAX_TARGETS = 12

    def guess_mode(name):
        # 'paper_syn_' veya 'synthetic_' -> synthetic; aksi -> real
        base = Path(name).stem
        return "synthetic" if (base.startswith("paper_syn_") or base.startswith("synthetic_")) else "real"

    def default_cfg(path):
        base = Path(path).stem
        mode = guess_mode(path)
        # Varsayılan genişlik/derinlik: synthetic -> (narrow,5); real -> (wide,11)
        if mode == "synthetic":
            return (path, mode, "narrow", 5)
        else:
            # bazı gerçek setlere daha düşük derinlik seçmek isteyebilirsin; şimdilik 11 tutuyoruz
            return (path, mode, "wide", 11)

    # Filtre: sadece ilgi çekici çiftler ve paper_syn_* ekle (çok büyümeyi engelle)
    keep = []
    prefer = re.compile(r"^(paper_syn_|synthetic_[ABC]_.*|mnist_1v7|fmnist_shirt_v_coat|svhn_3v8|cifar10_cat_v_dog|cifar10_car_v_truck)$")
    for p in npzs:
        stem = Path(p).stem
        if prefer.match(stem):
            keep.append(default_cfg(p))
    # Yedek: bulunamazsa rastgele birkaç tane daha ekle
    if len(keep) == 0:
        keep = [default_cfg(p) for p in npzs[:MAX_TARGETS]]
    targets = keep[:MAX_TARGETS]

# ------------------------------------------------------------------
board = ProgressBoard(
    columns=["dataset","mode","width","depth","seed","test_acc","best_k","rho","z","elapsed_s","run_id"],
    title="Stability (5 seeds)", total=len(targets)*5
)

rows=[]
for ds,mode,w,d in targets:
    for seed in range(5):
        t0=time.time()
        # Eğitim
        acc, run_id, lnames = run_one(ds, w, d, epochs=60, seed=seed)

        # k-süpürme: synthetic için ss=None; real için boyuta göre örneklem sayısı
        # (cifar/svhn için 1200; mnist/fmnist için 800; gerekirse ayarla)
        stem = Path(ds).stem
        if mode == 'synthetic':
            ss = None
        else:
            ss = 1200 if stem.startswith(("cifar10","svhn")) else 800

        # 0.99 bağlantı eşiği makale uyumlu; hız için auto_k_sweep_fast
        ((res,best), _) = timed_step(auto_k_sweep_fast, run_id, lnames, mode, ss, 0.99, seed)

        row = dict(
            dataset=stem, mode=mode, width=w, depth=d, seed=seed,
            test_acc=acc,
            best_k=(int(best[0]) if best else None),
            rho=(float(best[1]) if best else None) if best else None,
            z=(float(best[2]) if best else None) if best else None,
            elapsed_s=round(time.time()-t0,1), run_id=run_id
        )
        rows.append(row); board.add(**row)

# Özet tablo
df = pd.DataFrame(rows)
summary = df.groupby(["dataset","width","depth"]).agg(
    acc_mean=("test_acc","mean"), acc_std=("test_acc","std"),
    rho_mean=("rho","mean"),  rho_std=("rho","std"),
    z_mean=("z","mean"),      z_std=("z","std")
).reset_index()
summary["acc (μ±σ)"] = summary.apply(lambda r: f"{r.acc_mean:.4f} ± {r.acc_std:.4f}", axis=1)
summary["rho (μ±σ)"] = summary.apply(lambda r: f"{r.rho_mean:.4f} ± {r.rho_std:.4f}", axis=1)
summary["z (μ±σ)"]   = summary.apply(lambda r: f"{r.z_mean:.4f} ± {r.z_std:.4f}", axis=1)

from IPython.display import display
display(summary[["dataset","width","depth","acc (μ±σ)","rho (μ±σ)","z (μ±σ)"]])


In [ ]:
#@title Synthetic panel (Moons + Paper A/B/C) — ortak döngü
from pathlib import Path
import numpy as np, pandas as pd, time
from sklearn.datasets import make_moons

# -----------------------------
# Ortak yardımcılar
DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
RNG = np.random.default_rng(42)

def _save_npz_generic(Xtr, ytr, Xte, yte, name):
    p = DATA_DIR/f"{name}.npz"
    np.savez_compressed(p, X_train=Xtr, y_train=ytr, X_test=Xte, y_test=yte)
    return str(p)

def _split_balanced(X0, X1, n_train_each=1000, n_test_each=1000, rng=RNG):
    i0 = rng.permutation(len(X0)); i1 = rng.permutation(len(X1))
    X0_tr, X0_te = X0[i0[:n_train_each]], X0[i0[n_train_each:n_train_each+n_test_each]]
    X1_tr, X1_te = X1[i1[:n_train_each]], X1[i1[n_train_each:n_train_each+n_test_each]]
    Xtr = np.vstack([X0_tr, X1_tr]); ytr = np.hstack([np.zeros(len(X0_tr),dtype=np.int64), np.ones(len(X1_tr),dtype=np.int64)])
    Xte = np.vstack([X0_te, X1_te]); yte = np.hstack([np.zeros(len(X0_te),dtype=np.int64), np.ones(len(X1_te),dtype=np.int64)])
    p_tr = rng.permutation(len(Xtr)); p_te = rng.permutation(len(Xte))
    return Xtr[p_tr], ytr[p_tr], Xte[p_te], yte[p_te]

# -----------------------------
# Dataset oluşturucular (NPZ üretir -> path döner)

def build_moons_npz(noise, seed=0):
    rng_local = np.random.default_rng(123+int(noise*1000))
    X, y = make_moons(n_samples=10000, noise=noise, random_state=seed)
    idx = rng_local.permutation(len(X)); tr, te = idx[:1000], idx[1000:2000]
    p = DATA_DIR/f"moons_noise_{noise:.2f}.npz"
    np.savez_compressed(p, X_train=X[tr], y_train=y[tr], X_test=X[te], y_test=y[te])
    return str(p)

def build_sin_bands_npz(offset=0.9, noise=0.12, seed=0, n_each=4000):
    rng_local = np.random.default_rng(10+int(noise*1000))
    xs0 = rng_local.uniform(-np.pi, np.pi, size=n_each)
    xs1 = rng_local.uniform(-np.pi, np.pi, size=n_each)
    y0 = np.sin(xs0) - offset + rng_local.normal(0, noise, size=n_each)
    y1 = np.sin(xs1) + offset + rng_local.normal(0, noise, size=n_each)
    X0 = np.stack([xs0, y0], axis=1); X1 = np.stack([xs1, y1], axis=1)
    Xtr, ytr, Xte, yte = _split_balanced(X0, X1)
    name = f"paper_syn_A_sin_bands_off{offset:.2f}_nz{noise:.2f}"
    return _save_npz_generic(Xtr, ytr, Xte, yte, name)

def build_two_spirals_npz(noise=0.35, turns=3.5, seed=0, n_each=4000):
    rng_local = np.random.default_rng(20+int(noise*1000))
    theta = rng_local.uniform(0.0, turns*np.pi, size=n_each)
    a = 0.5; r = a*theta
    x0 = r*np.cos(theta) + rng_local.normal(0, noise, size=n_each)
    y0 = r*np.sin(theta) + rng_local.normal(0, noise, size=n_each)
    theta2 = theta + np.pi; r2 = a*theta2
    x1 = r2*np.cos(theta2) + rng_local.normal(0, noise, size=n_each)
    y1 = r2*np.sin(theta2) + rng_local.normal(0, noise, size=n_each)
    X0 = np.stack([x0, y0], axis=1); X1 = np.stack([x1, y1], axis=1)
    Xtr, ytr, Xte, yte = _split_balanced(X0, X1)
    name = f"paper_syn_B_two_spirals_nz{noise:.2f}_t{turns:.1f}"
    return _save_npz_generic(Xtr, ytr, Xte, yte, name)

def build_sine_surface_vs_plane_npz(noise_z=0.08, noise_xy=0.03, seed=0, n_each=6000, xy_range=(-3.0,3.0)):
    rng_local = np.random.default_rng(30+int(noise_z*1000))
    # Plane
    x0 = rng_local.uniform(xy_range[0], xy_range[1], size=n_each)
    y0 = rng_local.uniform(xy_range[0], xy_range[1], size=n_each)
    z0 = rng_local.normal(0, noise_z, size=n_each)
    X0 = np.stack([x0 + rng_local.normal(0, noise_xy, size=n_each),
                   y0 + rng_local.normal(0, noise_xy, size=n_each),
                   z0], axis=1)
    # Sine surface
    x1 = rng_local.uniform(xy_range[0], xy_range[1], size=n_each)
    y1 = rng_local.uniform(xy_range[0], xy_range[1], size=n_each)
    z1 = (np.sin(x1) + 0.5*np.sin(y1)) + rng_local.normal(0, noise_z, size=n_each)
    X1 = np.stack([x1 + rng_local.normal(0, noise_xy, size=n_each),
                   y1 + rng_local.normal(0, noise_xy, size=n_each),
                   z1], axis=1)
    Xtr, ytr, Xte, yte = _split_balanced(X0, X1)
    name = f"paper_syn_C_sine_surface_plane_nz{noise_z:.2f}_nxy{noise_xy:.2f}"
    return _save_npz_generic(Xtr, ytr, Xte, yte, name)

# -----------------------------
# Panel işler (sweep tanımı)
moons_noises = [0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45]
sinband_noises = [0.08,0.10,0.12,0.15]
spiral_noises = [0.20,0.30,0.35,0.45]
surface_noisez = [0.05,0.08,0.12,0.16]

jobs = [
    dict(dataset="moons",        param="noise",   values=moons_noises,
         builder=lambda v: build_moons_npz(v)),
    dict(dataset="paper_syn_A",  param="noise",   values=sinband_noises,
         builder=lambda v: build_sin_bands_npz(noise=v)),
    dict(dataset="paper_syn_B",  param="noise",   values=spiral_noises,
         builder=lambda v: build_two_spirals_npz(noise=v)),
    dict(dataset="paper_syn_C",  param="noise_z", values=surface_noisez,
         builder=lambda v: build_sine_surface_vs_plane_npz(noise_z=v)),
]

total_runs = sum(len(j["values"]) for j in jobs)

board = ProgressBoard(
    columns=["dataset","param","value","test_acc","best_k","rho","z","elapsed_s","run_id"],
    title="Synthetic sweep panel (Moons + Paper A/B/C)", total=total_runs
)

rows=[]
for job in jobs:
    for val in job["values"]:
        t0=time.time()
        ds_path = job["builder"](val)  # NPZ üret
        acc, rid, lns = run_one(ds_path, "narrow", 5, epochs=40, seed=0)  # hız için 40 epoch
        (res, best) = auto_k_sweep_fast(rid, lns, "synthetic", sample_size=None, rng_seed=0)
        row = dict(dataset=job["dataset"], param=job["param"], value=float(val),
                   test_acc=acc, best_k=(int(best[0]) if best else None),
                   rho=(float(best[1]) if best else None), z=(float(best[2]) if best else None),
                   elapsed_s=round(time.time()-t0,1), run_id=rid)
        rows.append(row)
        board.add(**row)

df = pd.DataFrame(rows).sort_values(by=["dataset","param","value"]).reset_index(drop=True)
display(df[["dataset","param","value","test_acc","rho","best_k","z"]])


In [ ]:
#@title CIFAR/SVHN + (paper_syn & synthetic discovered) k-grid rafinesi (panel)
import time, json, re
import numpy as np
from pathlib import Path

# --- Varsayılan/override parametreleri ---
def infer_mode(name):
    # 'real' (görüntü tabanlı) vs 'synthetic' (2D/3D nokta bulutu)
    if any(prefix in name for prefix in ["cifar", "svhn", "mnist", "fmnist"]):
        return "real"
    return "synthetic"

def infer_width_depth(name):
    # Basit sezgisel varsayılanlar + override
    if name.startswith("cifar10_cat_v_dog") or name.startswith("cifar10_car_v_truck"):
        return ("wide", 11)
    if name.startswith("svhn_"):
        return ("narrow", 5)
    # sentetikler: hızlı koşu için dar ve sığ
    return ("narrow", 5)

def suggest_k_grid(name, N):
    # Bilinen override'lar
    if name.startswith("cifar10_cat_v_dog"):
        return [90,120,150,180,200,220,250,300,350,400,500]
    if name.startswith("svhn_3v8"):
        return [10,20,30,50,90,120,150,180,200,220,250,300,350]
    # Genel sezgisel: küçük N için daha dar k aralığı
    if N <= 4000:
        return [5,10,15,20,30,40,50,60,80,100]
    else:
        return [10,20,30,50,90,120,150,180,200,220,250,300]

def choose_sample_size(N):
    # Büyük görüntü setleri için hız: 1200; küçük set için N'in yarısı üst sınır
    return int(min(1200, max(600, 0.6*N)))

# --- Var olan iki hedefi KORU ---
targets = [
    ("data/cifar10_cat_v_dog.npz","real","wide",11,[90,120,150,180,200,220,250,300,350,400,500],1200),
    ("data/svhn_3v8.npz","real","narrow",5,[10,20,30,50,90,120,150,180,200,220,250,300,350],1200),
]

# --- data/ içinden yeni NPZ'leri keşfet ve hedeflere EKLE ---
data_dir = Path("data")
known_keep = {Path(t[0]).name for t in targets}  # korunanların adı
if data_dir.exists():
    for npz_path in sorted(data_dir.glob("*.npz")):
        name = npz_path.stem
        if name in known_keep:
            continue  # zaten listede
        # Yalnızca makale ve sentetik eklemeleri hedefle (MNIST/fMNIST de dahil olabilir)
        if any(p in name for p in [
            "paper_syn_A_sin_bands_1k_each",
            "paper_syn_B_two_spirals_1k_each",
            "paper_syn_C_sine_surface_vs_plane_1k_each",
            "synthetic_A_moons_1k_each",
            "synthetic_B_circles_1k_each",
            "synthetic_C_blobs_1k_each",
            "cifar10_car_v_truck",
            "svhn_1v7",
            "mnist_", "fmnist_"
        ]):
            # Veri boyutu öğren
            try:
                dat = np.load(npz_path)
                Xtr = dat["X_train"]; ytr = dat["y_train"]
                N = Xtr.shape[0]
            except Exception:
                # okunamazsa es geç
                continue
            mode = infer_mode(name)
            width, depth = infer_width_depth(name)
            ks = suggest_k_grid(name, N)
            ss = choose_sample_size(N)
            targets.append((str(npz_path), mode, width, depth, ks, ss))

# --- Panel kurulumu ---
board = ProgressBoard(
    columns=["dataset","width","depth","k_values","best_k","rho","z","elapsed_s","note"],
    title="k-grid refine", total=len(targets)
)

# --- Çalıştır ---
for ds,mode,w,d,ks,ss in targets:
    t0 = time.time()
    acc, rid, lns = run_one(ds, w, d, epochs=60, seed=0)
    (res,best) = auto_k_sweep_fast_custom(
        run_id=rid, layer_names=lns, mode=mode, k_values=ks,
        sample_size=ss, rng_seed=0
    )
    board.add(dataset=Path(ds).stem, width=w, depth=d, k_values=str(ks),
              best_k=(int(best[0]) if best else None),
              rho=(float(best[1]) if best else None) if best else None,
              z=(float(best[2]) if best else None) if best else None,
              elapsed_s=round(time.time()-t0,1),
              note=("refined" if best else "no-connected/NA"))


In [ ]:
#@title Report Collector (özet tablo + grafikler) — UPDATED
from pathlib import Path
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import json, time, os, re

ART = Path("report_artifacts"); ART.mkdir(exist_ok=True)
(ART / "per_dataset").mkdir(exist_ok=True, parents=True)

def _safe_read_csv(p):
    try:
        return pd.read_csv(p)
    except Exception:
        return None

# 1) Ana grid özeti
df = _safe_read_csv("ricci_summary.csv")
if df is None or df.empty:
    raise RuntimeError("ricci_summary.csv bulunamadı ya da boş. Grid runner veya tekil koşul çıktısı üretmelisin.")

# --- Uyum / eksik sütun dayanıklılığı ---
for col in ["dataset","test_acc","rho","z","best_k","connected_all_layers","mode","width","depth"]:
    if col not in df.columns:
        # 'mode' için çıkarım yapacağız, diğerlerinde NaN tutmak OK
        df[col] = np.nan

# 2) Temel temizleme
# sayısal dönüştürmeler (hata toleranslı)
for c in ["test_acc","rho","z","best_k","width","depth"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# mode çıkarımı yoksa/eksikse üret:
def infer_mode(name: str):
    n = (name or "").lower()
    # "real" veri kümeleri:
    real_keys = ["mnist","fmnist","fashion","cifar","svhn","tiny", "imagenet"]
    # "synthetic" ipuçları:
    syn_keys  = ["synthetic","paper_syn","spiral","sine","moons","circles","blobs"]
    if any(k in n for k in real_keys) and not any(k in n for k in syn_keys):
        return "real"
    if any(k in n for k in syn_keys):
        return "synthetic"
    # bilinmiyorsa dataset ismine göre kestirim
    return "synthetic" if not re.search(r"(mnist|cifar|svhn|imagenet)", n) else "real"

if "mode" not in df.columns or df["mode"].isna().all():
    df["mode"] = df["dataset"].astype(str).map(infer_mode)
else:
    # doldurulamayanları da tamamla
    df.loc[df["mode"].isna(), "mode"] = df.loc[df["mode"].isna(), "dataset"].astype(str).map(infer_mode)

# yeni yardımcı sütunlar
df["rho_abs"] = df["rho"].abs()
df["ok"] = df["best_k"].notna() & df["rho"].notna() & (df["connected_all_layers"]==True)

# 3) Dataset bazında özet (acc, rho, z ortalama)
g = df.groupby("dataset", dropna=False).agg(
    n=("dataset","size"),
    acc_mean=("test_acc","mean"),
    acc_std=("test_acc","std"),
    rho_mean=("rho","mean"),
    rho_std=("rho","std"),
    z_mean=("z","mean"),
    z_std=("z","std"),
    ok_ratio=("ok","mean"),
    k_med=("best_k","median"),
    k_min=("best_k","min"),
    k_max=("best_k","max"),
).reset_index().sort_values(["mode" if "mode" in df.columns else "dataset","dataset"])
# 'mode' sütununu rapora taşıyalım (left join)
g = g.merge(df[["dataset","mode"]].drop_duplicates(), on="dataset", how="left")
cols = ["dataset","mode","n","acc_mean","acc_std","rho_mean","rho_std","z_mean","z_std","ok_ratio","k_min","k_med","k_max"]
g = g[cols]
g.to_csv(ART/"dataset_summary.csv", index=False)
display(g)

# 4) Mode × dataset bazında k ölçeği (synthetic vs real)
kstats = df[df["best_k"].notna()].groupby(["mode","dataset"], dropna=False).agg(
    k_med=("best_k","median"),
    k_min=("best_k","min"),
    k_max=("best_k","max")
).reset_index().sort_values(["mode","dataset"])
kstats.to_csv(ART/"kstats_by_dataset.csv", index=False)
display(kstats)

# 5) Scatter: accuracy vs rho  (beklenen: iyi acc -> daha negatif rho)
plt.figure()
ok_mask = df["ok"].fillna(False)
plt.scatter(df.loc[ok_mask,"test_acc"], df.loc[ok_mask,"rho"], s=16, alpha=0.8)
plt.xlabel("test_acc"); plt.ylabel("rho")
plt.title("Accuracy vs ρ (only valid best_k & connected)")
plt.grid(True); plt.tight_layout()
plt.savefig(ART/"scatter_acc_vs_rho.png", dpi=150)
plt.show()

# 6) Histogram: rho ve z
for col in ["rho","z"]:
    plt.figure()
    v = df[col].dropna()
    plt.hist(v, bins=24)
    plt.title(f"Histogram of {col}")
    plt.tight_layout()
    plt.savefig(ART/f"hist_{col}.png", dpi=150)
    plt.show()

# 7) Pozitif rho / best=None anomalileri listesi (rapor için)
anom = df[(df["rho"].notna() & (df["rho"]>0)) | df["best_k"].isna()]\
       .sort_values(["dataset","mode","width","depth"])
anom_cols = [c for c in ["dataset","mode","width","depth","test_acc","best_k","rho","z","connected_all_layers"] if c in df.columns]
anom.to_csv(ART/"anomalies.csv", index=False)
display(anom[anom_cols])

# 8) Mode bazlı kutu grafikleri (ρ ve accuracy)
for metric in ["rho","test_acc"]:
    plt.figure(figsize=(6,4))
    data = [df.loc[df["mode"]=="synthetic", metric].dropna(),
            df.loc[df["mode"]=="real", metric].dropna()]
    labels = ["synthetic","real"]
    plt.boxplot(data, labels=labels, showfliers=False)
    plt.title(f"{metric} by mode")
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.savefig(ART/f"box_{metric}_by_mode.png", dpi=150)
    plt.show()

# 9) Her dataset için mini rapor görselleri (per-dataset klasörü)
by_ds = df.groupby("dataset")
for ds_name, sub in by_ds:
    ds_dir = ART / "per_dataset" / str(ds_name)
    ds_dir.mkdir(parents=True, exist_ok=True)
    valid = sub[sub["ok"].fillna(False)]

    # acc vs rho (tek dataset)
    plt.figure()
    plt.scatter(valid["test_acc"], valid["rho"], s=16, alpha=0.85)
    plt.xlabel("test_acc"); plt.ylabel("rho"); plt.grid(True, alpha=0.3)
    plt.title(f"{ds_name} — acc vs ρ (valid only)")
    plt.tight_layout()
    plt.savefig(ds_dir/"scatter_acc_vs_rho.png", dpi=150)
    plt.close()

    # k dağılımı
    if valid["best_k"].notna().any():
        plt.figure()
        plt.hist(valid["best_k"].dropna(), bins=20)
        plt.xlabel("best_k"); plt.ylabel("count"); plt.grid(True, alpha=0.3)
        plt.title(f"{ds_name} — best_k distribution")
        plt.tight_layout()
        plt.savefig(ds_dir/"hist_best_k.png", dpi=150)
        plt.close()

    # rho ve z histogramları
    for col in ["rho","z"]:
        v = valid[col].dropna()
        if len(v) > 0:
            plt.figure()
            plt.hist(v, bins=20)
            plt.title(f"{ds_name} — hist of {col}")
            plt.tight_layout()
            plt.savefig(ds_dir/f"hist_{col}.png", dpi=150)
            plt.close()

# 10) Küçük JSON özet (rapor şablonu için)
payload = {
    "generated_at": time.ctime(),
    "n_rows": int(len(df)),
    "datasets": sorted([str(x) for x in df["dataset"].dropna().unique().tolist()]),
    "synthetic_datasets": sorted([str(x) for x in df.loc[df["mode"]=="synthetic","dataset"].dropna().unique().tolist()]),
    "real_datasets": sorted([str(x) for x in df.loc[df["mode"]=="real","dataset"].dropna().unique().tolist()]),
    "acc_vs_rho_corr_all": float(df[["test_acc","rho"]].dropna().corr().iloc[0,1]) if df[["test_acc","rho"]].dropna().shape[0] > 2 else None,
    "acc_vs_rho_corr_valid": float(df.loc[df["ok"]==True, ["test_acc","rho"]].dropna().corr().iloc[0,1]) if df.loc[df["ok"]==True, ["test_acc","rho"]].dropna().shape[0] > 2 else None,
}
with open(ART/"quick_summary.json","w") as f:
    json.dump(payload, f, indent=2)
print("Saved artifacts ->", ART.resolve())
print("Per-dataset plots ->", (ART/"per_dataset").resolve())


In [ ]:
#@title Ricci Suite: MNIST ikilileri + Paper Synthetic A/B/C + mevcut synthetic/CIFAR/SVHN (panel + tablo + CSV)
# Guard: ProgressBoard/timed_step yoksa inline tanımla
try:
    ProgressBoard; timed_step
except NameError:
    from dataclasses import dataclass, field
    from typing import List, Dict, Any, Optional
    import time, pandas as pd, numpy as np
    from IPython.display import clear_output, display
    @dataclass
    class ProgressBoard:
        columns: List[str]
        title: str = "Progress"
        rows: List[Dict[str, Any]] = field(default_factory=list)
        total: Optional[int] = None
        t0: float = field(default_factory=time.time)
        def add(self, **kwargs):
            self.rows.append(kwargs); self.render()
        def render(self):
            clear_output(wait=True)
            df = pd.DataFrame(self.rows)
            for c in self.columns:
                if c not in df.columns: df[c] = np.nan
            df = df[self.columns]
            sort_cols = [c for c in ["dataset","width","depth","elapsed_s"] if c in df.columns]
            if sort_cols: df = df.sort_values(by=sort_cols, na_position='last')
            display(df)
            done = len(self.rows)
            if self.total:
                pct = 100.0 * done / self.total
                elapsed = time.time() - self.t0
                print(f"[{self.title}] {done}/{self.total} = {pct:.1f}% | elapsed {elapsed:.1f}s")
            else:
                print(f"[{self.title}] steps={done} | elapsed {time.time()-self.t0:.1f}s")
    def timed_step(fn, *args, **kwargs):
        t0 = time.time(); out = fn(*args, **kwargs); return out, round(time.time()-t0, 1)

# --- importlar & yardımcılar ---
from pathlib import Path
import re, time
import numpy as np, pandas as pd
from tensorflow.keras.datasets import mnist

DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)

def _subset_digits_flat(x, y, pos, neg):
    sel = np.where((y==pos)|(y==neg))[0]
    X = x[sel].astype('float32')/255.0
    yb = (y[sel]==pos).astype('int64')  # pos sınıf = 1
    X = X.reshape((X.shape[0], -1))     # 28x28 -> 784
    return X, yb

def ensure_mnist_pair(a:int, b:int)->str:
    """mnist_{a}v{b}.npz varsa atla; yoksa oluştur ve kaydet. path döndürür."""
    name = f"mnist_{a}v{b}"
    p = DATA_DIR / f"{name}.npz"
    if p.exists():
        return str(p)
    (Xtr, ytr), (Xte, yte) = mnist.load_data()
    Xtr2, ytr2 = _subset_digits_flat(Xtr, ytr, a, b)
    Xte2, yte2 = _subset_digits_flat(Xte, yte, a, b)
    np.savez_compressed(p, X_train=Xtr2, y_train=ytr2, X_test=Xte2, y_test=yte2)
    print("saved", p, Xtr2.shape, Xte2.shape)
    return str(p)

MNIST_PAIR_RE = re.compile(r"^mnist_(\d)v(\d)$")

def get_npz_path(name:str)->str:
    """
    - 'mnist_{a}v{b}' biçimi ise gerekirse üretir, path döndürür.
    - Diğer isimler için data/{name}.npz varlığını kontrol eder.
    """
    m = MNIST_PAIR_RE.match(name)
    if m:
        a, b = int(m.group(1)), int(m.group(2))
        return ensure_mnist_pair(a, b)
    p = DATA_DIR / f"{name}.npz"
    if not p.exists():
        raise FileNotFoundError(f"data/{name}.npz bulunamadı. Önce dataset hücresini çalıştır.")
    return str(p)

def infer_sample_size(npz_path:str, cap:int=1000)->int:
    """auto_k_sweep_fast için train boyutundan makul bir örnekleme büyüklüğü seç."""
    d = np.load(npz_path)
    n_train = int(d["X_train"].shape[0])
    return int(min(cap, n_train))

# --- hedef veri setleri ---
# MNIST ikilileri
MNIST_PAIRS = ["mnist_4v7","mnist_1v5","mnist_2v6","mnist_6v7"]

# Paper synthetic (bu isimler dataset hücresinde üretildi)
PAPER_SYNTHETIC = [
    "paper_syn_A_sin_bands_1k_each",
    "paper_syn_B_two_spirals_1k_each",
    "paper_syn_C_sine_surface_vs_plane_1k_each",
]

# Mevcut synthetic/CIFAR/SVHN (dataset hücresinin ürettikleri)
OPTIONAL_EXISTING = [
    "synthetic_A_moons_1k_each",
    "synthetic_B_circles_1k_each",
    "synthetic_C_blobs_1k_each",
    "cifar10_cat_v_dog",
    "cifar10_car_v_truck",
    "svhn_1v7",
    "svhn_3v8",
]

# Birleştir (mevcut olup olmadığına göre filtreleyelim)
ALL_DATASET_NAMES = []
ALL_DATASET_NAMES += MNIST_PAIRS
ALL_DATASET_NAMES += PAPER_SYNTHETIC
# OPTIONAL_EXISTING içinden data/ altında bulunanları ekle
for nm in OPTIONAL_EXISTING:
    if (DATA_DIR / f"{nm}.npz").exists():
        ALL_DATASET_NAMES.append(nm)

# --- mimari seti ---
WIDTHS = ["narrow","wide","bottleneck"]
DEPTHS = [5,11]

# --- panel hazırlığı ---
total = len(ALL_DATASET_NAMES)*len(WIDTHS)*len(DEPTHS)
board = ProgressBoard(
    columns=["dataset","width","depth","test_acc","best_k","rho","z",
             "connected_all_layers","elapsed_s","run_id","error"],
    title="Ricci Suite sweep", total=total
)

rows=[]
for name in ALL_DATASET_NAMES:
    try:
        ds_path = get_npz_path(name)
    except Exception as e:
        # Veri seti bulunamazsa hata satırı yazıp diğerlerine devam et
        board.add(dataset=name, error=str(e), elapsed_s=np.nan)
        continue

    for w in WIDTHS:
        for d in DEPTHS:
            t0 = time.time()
            try:
                # ---- Train ----
                (train_out, train_sec) = timed_step(run_one, ds_path, w, d, 60, 128, 0)
                acc, run_id, layer_names = train_out

                # ---- Fast Ricci / k-sweep ----
                sample_size = infer_sample_size(ds_path, cap=1000)  # otomatik örneklem büyüklüğü
                ((res, best), sweep_sec) = timed_step(
                    auto_k_sweep_fast, run_id, layer_names, "real", sample_size, 0.99, 0
                )

                row = dict(
                    dataset=name, width=w, depth=d,
                    test_acc=float(acc),
                    best_k=(int(best[0]) if best else None),
                    rho=(float(best[1]) if best else None) if best else None,
                    z=(float(best[2]) if best else None) if best else None,
                    connected_all_layers=(bool(best[-1]) if best else False) if best else False,
                    elapsed_s=round(time.time()-t0,1),
                    run_id=run_id
                )
                rows.append(row); board.add(**row)

            except Exception as e:
                row = dict(dataset=name, width=w, depth=d, error=str(e),
                           elapsed_s=round(time.time()-t0,1))
                rows.append(row); board.add(**row)

# -- final tablo & kaydet --
df_suite = pd.DataFrame(rows).sort_values(by=["dataset","width","depth"], na_position='last')
csv_out = Path("ricci_suite_summary.csv"); df_suite.to_csv(csv_out, index=False)
print("Saved ->", csv_out)
df_suite


In [ ]:
#@title MLP-Arch + OPT/LR/LOSS destekli run_one_arch_opt (Keras 3 uyumlu) **UPDATED: dataset discovery + batch runner**
# Guard: ProgressBoard yoksa inline tanımla
try:
    ProgressBoard; timed_step
except NameError:
    from dataclasses import dataclass, field
    from typing import List, Dict, Any, Optional
    import time, pandas as pd, numpy as np
    from IPython.display import clear_output, display
    @dataclass
    class ProgressBoard:
        columns: List[str]; title: str = "Progress"
        rows: List[Dict[str, Any]] = field(default_factory=list)
        total: Optional[int] = None; t0: float = field(default_factory=time.time)
        def add(self, **kwargs): self.rows.append(kwargs); self.render()
        def render(self):
            clear_output(wait=True)
            df = pd.DataFrame(self.rows)
            for c in self.columns:
                if c not in df.columns: df[c] = np.nan
            df = df[self.columns]
            sort_cols = [c for c in ["dataset","width","depth","elapsed_s"] if c in df.columns]
            if sort_cols: df = df.sort_values(by=sort_cols, na_position='last')
            display(df)
            done = len(self.rows)
            if self.total:
                pct = 100.0 * done / self.total; elapsed = time.time()-self.t0
                print(f"[{self.title}] {done}/{self.total} = {pct:.1f}% | elapsed {elapsed:.1f}s")
            else:
                print(f"[{self.title}] steps={done} | elapsed {time.time()-self.t0:.1f}s")
    def timed_step(fn, *args, **kwargs):
        t0=time.time(); out=fn(*args, **kwargs); return out, round(time.time()-t0,1)

from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

EMB_DIR = Path("embeddings"); EMB_DIR.mkdir(parents=True, exist_ok=True)
CKPT_DIR = Path("checkpoints"); CKPT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path("data"); DATA_DIR.mkdir(parents=True, exist_ok=True)
RUNS_CSV = Path("runs_summary.csv")

# -------- activation / optimizer / loss helpers --------
def get_activation_layer(name: str):
    name = name.lower()
    if name=="relu":  return layers.ReLU()
    if name=="gelu":  return layers.Activation(tf.keras.activations.gelu)
    if name in ("silu","swish"): return layers.Activation(tf.keras.activations.silu)
    if name=="tanh":  return layers.Activation("tanh")
    if name=="leakyrelu": return layers.LeakyReLU(alpha=0.1)
    if name=="elu":   return layers.ELU(alpha=1.0)
    raise ValueError(f"Unsupported activation: {name}")

def get_optimizer(name: str, lr: float):
    name = name.lower()
    if name=="rmsprop": return keras.optimizers.RMSprop(learning_rate=lr)
    if name=="adam":    return keras.optimizers.Adam(learning_rate=lr)
    if name=="adamw":   return keras.optimizers.AdamW(learning_rate=lr, weight_decay=1e-4)
    if name=="sgd":     return keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    raise ValueError(f"Unsupported optimizer: {name}")

def focal_loss(alpha=0.25, gamma=2.0, label_smoothing=0.0):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        if label_smoothing>0:
            y_true = y_true*(1-label_smoothing) + 0.5*label_smoothing
        eps = 1e-7
        y_pred = tf.clip_by_value(y_pred, eps, 1-eps)
        p_t = y_true*y_pred + (1-y_true)*(1-y_pred)
        alpha_t = y_true*alpha + (1-y_true)*(1-alpha)
        return tf.reduce_mean(-alpha_t * tf.pow(1.0 - p_t, gamma) * tf.math.log(p_t))
    return loss

def make_hinge_loss(squared=False):
    # Bizim etiketler 0/1; hinge {-1,+1} ister. Dönüştürüyoruz.
    def loss(y_true, y_pred):
        y = tf.cast(y_true, tf.float32)*2.0 - 1.0
        s = (y_pred*2.0 - 1.0)  # sigmoid çıktı -> skora dönüştür
        m = 1.0 - y*s
        if squared:
            m = tf.square(tf.maximum(0.0, m))
        else:
            m = tf.maximum(0.0, m)
        return tf.reduce_mean(m)
    return loss

def get_loss(name: str):
    name = name.lower()
    if name in ("bce","binary_crossentropy"): return "binary_crossentropy"
    if name=="focal": return focal_loss(alpha=0.25, gamma=2.0)
    if name=="hinge": return make_hinge_loss(squared=False)
    if name in ("squared_hinge","hinge2"): return make_hinge_loss(squared=True)
    raise ValueError(f"Unsupported loss: {name}")

def _sanitize(s: str) -> str:
    return s.replace("/","_").replace("\\","_").replace(" ","_")

# -------- model builder --------
def build_mlp_arch(input_dim, width="narrow", depth=5,
                   activation="relu", batchnorm=False, dropout=0.0, residual=False):
    assert width in ["narrow","wide","bottleneck"]
    if residual and width=="bottleneck":
        residual=False  # boyut değişimi nedeniyle güvenli değil

    inputs = keras.Input(shape=(input_dim,))
    x = inputs; taps=[]; names=[]

    def block(x, units, i):
        z = layers.Dense(units, use_bias=(not batchnorm), name=f"dense_{i}_{units}")(x)
        if batchnorm: z = layers.BatchNormalization(name=f"bn_{i}")(z)
        z = get_activation_layer(activation)(z)
        if dropout and dropout>0: z = layers.Dropout(dropout, name=f"drop_{i}")(z)
        if residual and (x.shape[-1]==units):
            z = layers.Add(name=f"res_{i}")([x, z])
        taps.append(z); names.append(f"block{i}_{units}_{activation}{'_bn' if batchnorm else ''}{f'_do{dropout}' if dropout else ''}{'_res' if residual else ''}")
        return z

    if width=="bottleneck":
        units_list = [50,50,25,50,50] if depth==5 else [50,50,50,50,50,25,50,50,50,50,50]
    else:
        units_list = [25 if width=="narrow" else 50]*depth

    for i,u in enumerate(units_list,1): x = block(x,u,i)
    out = layers.Dense(1, activation="sigmoid", name="out")(x)
    model = keras.Model(inputs, out)
    inter = keras.Model(inputs, taps)
    return model, inter, names

# -------- single run --------
def run_one_arch_opt(dataset_npz, width, depth,
                     activation="relu", batchnorm=False, dropout=0.0, residual=False,
                     optimizer="rmsprop", lr=1e-3, loss="bce",
                     epochs=60, batch_size=128, seed=0,
                     standardize_small_dim=True):
    d = np.load(dataset_npz)
    Xtr,ytr,Xte,yte = d['X_train'], d['y_train'], d['X_test'], d['y_test']
    # Opsiyonel: küçük boyutlu sentetiklerde standardizasyon
    if standardize_small_dim and Xtr.shape[1] <= 10:
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler().fit(Xtr)
        Xtr = sc.transform(Xtr); Xte = sc.transform(Xte)

    tf.keras.utils.set_random_seed(seed)
    model, inter, layer_names = build_mlp_arch(
        Xtr.shape[1], width, depth, activation, batchnorm, dropout, residual
    )
    opt  = get_optimizer(optimizer, lr)
    los  = get_loss(loss)
    model.compile(optimizer=opt, loss=los, metrics=["accuracy"])

    _ = model.fit(Xtr, ytr, validation_data=(Xte, yte), epochs=epochs, batch_size=batch_size, verbose=0)
    te_loss, te_acc = model.evaluate(Xte, yte, verbose=0)
    emb_list = inter.predict(Xte, batch_size=1024, verbose=0)

    base = (
        Path(dataset_npz).stem
        + f"__{width}_d{depth}_act{activation}"
        + (f"_bn" if batchnorm else "")
        + (f"_do{dropout}" if dropout else "")
        + (f"_res" if residual else "")
        + f"__opt{optimizer}_lr{lr:g}_loss{loss}"
        + f"_seed{seed}"
    )
    out_dir = EMB_DIR / base; out_dir.mkdir(parents=True, exist_ok=True)
    for name, E in zip(layer_names, emb_list):
        np.save(out_dir / f"{_sanitize(name)}.npy", E)
    model.save(CKPT_DIR / f"{base}.keras")
    np.save(out_dir/"y_test.npy", yte); np.save(out_dir/"X_test.npy", Xte)
    return float(te_acc), base, layer_names

# -------- dataset discovery + batch cycle --------
def list_npz_datasets(patterns=None, exclude_patterns=None):
    """data/ altındaki NPZ dosyalarını listeler; paper_syn_* dahil."""
    files = sorted(DATA_DIR.glob("*.npz"))
    names = [f.name for f in files]
    if patterns:
        keep = []
        for n in names:
            if any(p in n for p in patterns):
                keep.append(n)
        names = keep
    if exclude_patterns:
        names = [n for n in names if not any(p in n for p in exclude_patterns)]
    return [DATA_DIR / n for n in names]

def run_batch(datasets, grid, epochs=60, batch_size=128, seed=0, board_title="Runs"):
    cols


In [ ]:
#@title Training Config Sweep: OPT/LR/LOSS + ACT/BN/DO/RES (panel + CSV)
from pathlib import Path
import numpy as np, pandas as pd, time

DATASETS = [
  # --- mevcutlar ---
  ("data/synthetic_A_moons_1k_each.npz","synthetic"),
  ("data/mnist_1v7.npz","real"),
  ("data/fmnist_sandal_v_boot.npz","real"),
  ("data/cifar10_car_v_truck.npz","real"),
  ("data/svhn_1v7.npz","real"),

  # --- EKLENDİ: makaledeki sentetik A/B/C ---
  ("data/paper_syn_A_sin_bands_1k_each.npz","synthetic"),
  ("data/paper_syn_B_two_spirals_1k_each.npz","synthetic"),
  ("data/paper_syn_C_sine_surface_vs_plane_1k_each.npz","synthetic"),
]

WIDTHS = ["narrow","wide"]
DEPTHS = [5,11]
ACTS   = ["relu","gelu","silu"]          # istersen "tanh" ekle
BNS    = [False, True]
DROPS  = [0.0, 0.2]
RESID  = [False]

OPTS   = ["rmsprop","adam"]              # istersen "adamw","sgd" ekle
LRS    = [1e-3, 3e-4]                    # istersen [1e-3,5e-4,1e-4]
LOSSES = ["bce","focal"]                 # istersen "hinge","squared_hinge" ekle

def _sample_size_for(mode, ds_name):
    # sentetiklerde tamamını kullan; büyük gerçek setlerde hızlı tarama için örnekleme
    if mode=='synthetic': 
        return None
    return 1200 if ds_name.startswith(("cifar10","svhn")) else 800

total = (len(DATASETS)*len(WIDTHS)*len(DEPTHS)*len(ACTS)*
         len(BNS)*len(DROPS)*len(RESID)*len(OPTS)*len(LRS)*len(LOSSES))

board = ProgressBoard(
    columns=["dataset","mode","width","depth","act","bn","drop","res",
             "opt","lr","loss","test_acc","best_k","rho","z",
             "connected_all_layers","elapsed_s","run_id","error"],
    title="Training Config Sweep", total=total
)

rows=[]
for (ds, mode) in DATASETS:
    ds_name = Path(ds).stem
    for w in WIDTHS:
        for d in DEPTHS:
            for act in ACTS:
                for bn in BNS:
                    for dr in DROPS:
                        for rs in RESID:
                            for opt in OPTS:
                                for lr in LRS:
                                    for loss in LOSSES:
                                        t0=time.time()
                                        try:
                                            # Train + embed
                                            (train_out, train_secs) = timed_step(
                                                run_one_arch_opt, ds, w, d,
                                                act, bn, dr, rs,
                                                opt, lr, loss,
                                                60, 128, 0
                                            )
                                            acc, run_id, lns = train_out

                                            # k-sweep
                                            ss = _sample_size_for(mode, ds_name)
                                            ((res, best), sweep_secs) = timed_step(
                                                auto_k_sweep_fast, run_id, lns, mode, ss, 0.99, 0
                                            )

                                            row = dict(
                                                dataset=ds_name, mode=mode, width=w, depth=d,
                                                act=act, bn=bn, drop=dr, res=rs,
                                                opt=opt, lr=lr, loss=loss,
                                                test_acc=float(acc),
                                                best_k=(int(best[0]) if best else None),
                                                rho=(float(best[1]) if best else None) if best else None,
                                                z=(float(best[2]) if best else None) if best else None,
                                                connected_all_layers=(bool(best[-1]) if best else False) if best else False,
                                                elapsed_s=round(time.time()-t0,1), run_id=run_id
                                            )
                                            rows.append(row); board.add(**row)

                                        except Exception as e:
                                            row = dict(dataset=ds_name, mode=mode, width=w, depth=d,
                                                       act=act, bn=bn, drop=dr, res=rs,
                                                       opt=opt, lr=lr, loss=loss,
                                                       error=str(e), elapsed_s=round(time.time()-t0,1))
                                            rows.append(row); board.add(**row)

df_cfg = pd.DataFrame(rows).sort_values(
    by=["dataset","width","depth","act","bn","drop","res","opt","lr","loss"],
    na_position='last'
)
out_csv = Path("ricci_training_config_sweep.csv"); df_cfg.to_csv(out_csv, index=False)
print("Saved ->", out_csv)
df_cfg
